In [5]:
from dataclasses import dataclass
import numpy as np
from typing import List, Tuple
from scipy.stats import spearmanr

In [6]:
class Player:
    def __init__(self, num_players):
        self.scores = np.zeros(num_players)
        self.player_scores = np.zeros((num_players, num_players))
        self.player_alive = np.ones(num_players)

    # カミングアウトする関数
    def co(self):
        assert NotImplementedError

    # 投票する関数
    def vote(self):
        assert NotImplementedError

    # 推理を披露する関数
    def tell_score(self):
        assert NotImplementedError

    # 推理する関数
    def infer(self, co_list: List[str], score_list: List[np.array]):
        assert NotImplementedError

    # 夜時間のアクションを行う関数
    def action(self):
        assert NotImplementedError

class Villager(Player):
    def __init__(self, num_players):
        super().__init__(num_players)

    def co(self):
        return "VILLAGER"

    def vote(self):
        # 生存者の中で最もscoreが高いプレイヤーに投票
        max_score = -np.inf
        max_idx = -1
        for i in range(len(self.scores)):
            if self.player_alive[i] == 1 and self.scores[i] > max_score:
                max_score = self.scores[i]
                max_idx = i
        return max_idx

    def tell_score(self):
        return self.scores

    def infer(self, co_list: List[str], score_list: List[np.array]):
        corr = [float(spearmanr(self.scores, s)[0]) for s in score_list]
        self.scores = [
            self.scores[i] + corr[i] for i in range(len(score_list))
        ]

    def action(self):
        pass

class Werewolf(Player):
    def __init__(self, num_players):
        super().__init__(num_players)

    def co(self):
        # ランダムにカミングアウト
        if np.

    def vote(self):
        # 生存者の中で最もscoreが低いプレイヤーに投票
        min_score = np.inf
        min_idx = -1
        for i in range(len(self.scores)):
            if self.player_alive[i] == 1 and self.scores[i] < min_score:
                min_score = self.scores[i]
                min_idx = i
        return min_idx

    def tell_score(self):
        return self.scores

    def infer(self, co_list: List[str], score_list: List[np.array]):
        corr = [float(spearmanr(self.scores, s)[0]) for s in score_list]
        self.scores = [
            self.scores[i] + corr[i] for i in range(len(score_list))
        ]

    def action(self):
        pass

In [4]:
score_matrix = [np.random.rand(5) for _ in range(5)]
score = score_matrix[0]
# スピアマンの順位相関係数を計算
correlation = [float(spearmanr(score, s)[0]) for s in score_matrix]
correlation

[0.9999999999999999, 0.3, -0.6, -0.7, -0.19999999999999998]

In [16]:
correlation[0].value

AttributeError: 'numpy.float64' object has no attribute 'value'

In [12]:
score_matrix = [np.random.rand(5) for _ in range(5)]
# 順位相関係数
spearmanr(score_matrix)[0], score_matrix[:2]

(array([[ 1. , -0.1,  0.1,  0.6,  0.5],
        [-0.1,  1. , -0.8, -0.3,  0.2],
        [ 0.1, -0.8,  1. ,  0. , -0.2],
        [ 0.6, -0.3,  0. ,  1. ,  0.8],
        [ 0.5,  0.2, -0.2,  0.8,  1. ]]),
 [array([0.57149901, 0.32498221, 0.46287464, 0.9752086 , 0.62584837]),
  array([0.40575427, 0.91892143, 0.13521057, 0.20102074, 0.51365573])])

In [3]:
import random

In [12]:
class Game:
    def __init__(self, roles):
        self.day = 0
        self.is_gameover = False
        self.winners = None
        self._define_players(roles)

    def _define_roles(self, roles: List[str], seed: int = 0):
        num_players = len(roles)
        roles = random.sample(roles, len(roles), seed)
        self.players = []
        for role in roles:
            if role == "village":
                player = Village(
                    not_werevolf_player=random.choice([i for i in range(num_players) if roles[i] != "werewolf"], seed)
                )
            elif role == "werewolf":
                player = Werewolf(
                    werewolf_players=[i for i in range(num_players) if roles[i] == "werewolf"]
                )
            elif role == "seer":
                player = Seer()
            elif role == "bodyguard":
                player = Bodyguard()
            else:
                raise ValueError("Invalid role")
            self.players.append(player)

    def run(self):
        while not self.is_gameover:
            self.day += 1
            self._meeting()
            self._vote()
            self._judge()
            if self.is_gameover:
                break
            self._action()
            self._judge()
        return self.winners

    # 会議を行う関数
    def _meeting(self):
        self.co_list = [player.co() for player in self.players]
        self.scores = np.array([player.tell_score() for player in self.players])
        
        for player in self.players:
            player.infer(self.co_list, self.scores)
    
    # 投票を行う関数
    def _vote(self):
        self.votes = np.array([player.vote() for player in self.players])
        self.vote_result = np.argmax(np.bincount(self.votes))
        self.players[self.vote_result] = False

    # 夜時間のアクションを行う関数
    def _action(self):
        werewolf_target = []
        bodyguard_target = None
        for player in self.players:
            if player.alive:
                if player.role == "village":
                    continue
                elif player.role == "werewolf":
                    werewolf_target.append(player.action())
                elif player.role == "seer":
                    seer_target = player.action()
                    assert(self.players[seer_target].alive)
                    if self.players[seer_target].role == "werewolf":
                        player.scores[seer_target] = np.inf
                    else:
                        player.scores[seer_target] = -np.inf
                elif player.role == "bodyguard":
                    bodyguard_target = player.action()
                    assert(self.players[bodyguard_target].alive)
            else:
                continue
        werewolf_target = np.random.choice(werewolf_target)
        if bodyguard_target != werewolf_target:
            self.players[werewolf_target].alive = False
    
    def _judge(self):
        if self.players.count("werewolf") == 0:
            self.is_gameover = True
            self.winners = "village"
        elif self.players.count("werewolf") >= len(self.players) / 2:
            self.is_gameover = True
            self.winners = "werewolf"
        else:
            self.is_gameover = False

In [5]:
assert(False)

AssertionError: 

In [24]:
np.bincount(np.array([1,2,3,4,5,3]))

array([0, 1, 1, 2, 1, 1])

In [ ]:
class Seer(Player):
    def __init__(self, game: Game):
        super().__init__(game)
    
    def co(self):
        return "SEER"
    
    def vote(self):
        return np.argmin(self.infer())
    
    def talk(self):
        return 
    
    def infer(self):
        return 0
    
    def action(self):
        action_target_player = self.determine_action_target_player()